In [115]:
import cv2
import pandas as pd
import numpy as np
import torch
from ultralytics import YOLO
from tracker import*
import time


In [116]:
model=YOLO('yolo12s.pt')

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

cuda


In [117]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

my_class = ['car', 'motorcycle', 'truck']

In [118]:
tracker=Tracker()
video=cv2.VideoCapture('road_vichele.mp4')
fps = video.get(cv2.CAP_PROP_FPS)

In [119]:
counter = []

frame_dict = {}
frame_no = 0

In [120]:
def search_frame(current, tragert_id):
    for frame in range(current, 0, -1):
        for data in frame_dict[frame]:
            id , cx, cy = data
            if(id == tragert_id):
                return [frame, cx, cy]
            
    return None

def distance(xs, ys, xe, ye):
    return np.sqrt((xe - xs)**2 + (ye - ys)**2)

In [121]:
# Define a speed threshold for overspeed warning
speed_threshold = 90  # km/h

# Initialize dictionaries to store total speed and count of vehicles for each class
speed_data = {'car': {'total_speed': 0, 'count': 0},
              'motorcycle': {'total_speed': 0, 'count': 0},
              'truck': {'total_speed': 0, 'count': 0}}

In [ ]:


while True:    
    ret, frame = video.read()
    frame_no += 1
    frame_dict[frame_no] = []

    if not ret:
        break

    frame = cv2.resize(frame, (540, 960))
  
    results = model.predict(frame)
    temp = results[0].boxes.data
    temp = temp.detach().cpu().numpy()
    bb_df = pd.DataFrame(temp).astype("int")

    bb_list = []
            
    for index, row in bb_df.iterrows():
        x1, y1, x2, y2, sc, d = row
        c = class_list[d]
        if c in my_class:  # Ensure that motorcycles, trucks, and cars are included
            bb_list.append([x1, y1, x2, y2, sc, c])

    bbox_ids = tracker.update(bb_list)

    for bbox in bbox_ids:
        x3, y3, x4, y4, sc, c, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        cv2.putText(frame, str(c), (x3, y3 - 2), cv2.QT_FONT_NORMAL, 0.3, (255, 255, 255), 1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 1)

        counter_line_y = 550
        speed_line_y = 500
        unit_def = 8
        offset = 5

        if counter_line_y < (cy + offset) and counter_line_y > (cy - offset): 
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            
            if id not in counter:
                counter.append(id)
                frame_dict[frame_no].append([id, cx, cy])

        if speed_line_y < (cy + offset) and speed_line_y > (cy - offset): 
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

            last_data = search_frame(frame_no, id)
            if (last_data is not None):
                last_frame, xs, ys = last_data
                d = distance(xs, ys, cx, cy) / unit_def
                t = (frame_no - last_frame) / fps
                s = d / t * 3.6

                # Update speed data for calculating averages
                if c in speed_data:
                    speed_data[c]['total_speed'] += s
                    speed_data[c]['count'] += 1

                # Display speed and overspeed warning
                cv2.putText(frame, f"{s:.2f} km/h", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
                if s > speed_threshold:
                    cv2.putText(frame, "OVERSPEED!", (cx, cy + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display vehicle count
    
    cv2.putText(frame, ('count - ') + str(len(counter)), (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)    

    # Display average speed for cars, trucks, and motorcycles
    cv2.putText(frame, f"Avg Car Speed: {speed_data['car']['total_speed'] / speed_data['car']['count']:.2f} km/h" if speed_data['car']['count'] > 0 else "Avg Car Speed: N/A", (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 0), 1)
    cv2.putText(frame, f"Avg Truck Speed: {speed_data['truck']['total_speed'] / speed_data['truck']['count']:.2f} km/h" if speed_data['truck']['count'] > 0 else "Avg Truck Speed: N/A", (60, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(frame, f"Avg Motorcycle Speed: {speed_data['motorcycle']['total_speed'] / speed_data['motorcycle']['count']:.2f} km/h" if speed_data['motorcycle']['count'] > 0 else "Avg Motorcycle Speed: N/A", (60, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    

    # Display average speed for each class
    y_offset = 60
    for vehicle_class, data in speed_data.items():
        if data['count'] > 0:
            avg_speed = data['total_speed'] / data['count']
            cv2.putText(frame, f"Avg {vehicle_class} Speed: {avg_speed:.2f} km/h", (60, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            y_offset += 20

    print(frame_dict[frame_no])
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

video.release()
cv2.destroyAllWindows()



0: 640x384 5 cars, 1 truck, 28.4ms
Speed: 5.7ms preprocess, 28.4ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 5 cars, 2 trucks, 18.9ms
Speed: 2.6ms preprocess, 18.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 5 cars, 1 truck, 23.1ms
Speed: 4.0ms preprocess, 23.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 4 cars, 1 truck, 33.7ms
Speed: 3.8ms preprocess, 33.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 4 cars, 1 truck, 33.9ms
Speed: 5.9ms preprocess, 33.9ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 4 cars, 1 truck, 38.2ms
Speed: 4.0ms preprocess, 38.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 5 cars, 1 truck, 27.1ms
Speed: 4.1ms preprocess, 27.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
[]

0: 640x384 5 cars, 1 truck, 27.5ms
Speed: 4.0m

KeyboardInterrupt: 